In [1]:
import cvxpy as cp
import pandas as pd
import numpy as np
import scipy

# Question 2: Maximum likelihood prediction of team ability 

### Convert Matlab data generation to numpy

In [2]:
n = 10
m = 45
m_test = 45
sigma = 0.250

train = np.array(
    [
        [1, 2, 1],
        [1, 3, 1],
        [1, 4, 1],
        [1, 5, 1],
        [1, 6, 1],
        [1, 7, 1],
        [1, 8, 1],
        [1, 9, 1],
        [1, 10, 1],
        [2, 3, -1],
        [2, 4, -1],
        [2, 5, -1],
        [2, 6, -1],
        [2, 7, -1],
        [2, 8, -1],
        [2, 9, -1],
        [2, 10, -1],
        [3, 4, 1],
        [3, 5, -1],
        [3, 6, -1],
        [3, 7, 1],
        [3, 8, 1],
        [3, 9, 1],
        [3, 10, 1],
        [4, 5, -1],
        [4, 6, -1],
        [4, 7, 1],
        [4, 8, 1],
        [4, 9, -1],
        [4, 10, -1],
        [5, 6, 1],
        [5, 7, 1],
        [5, 8, 1],
        [5, 9, -1],
        [5, 10, 1],
        [6, 7, 1],
        [6, 8, 1],
        [6, 9, -1],
        [6, 10, -1],
        [7, 8, 1],
        [7, 9, 1],
        [7, 10, -1],
        [8, 9, -1],
        [8, 10, -1],
        [9, 10, 1],
    ]
)

test = np.array(
    [
        [1, 2, 1],
        [1, 3, 1],
        [1, 4, 1],
        [1, 5, 1],
        [1, 6, 1],
        [1, 7, 1],
        [1, 8, 1],
        [1, 9, 1],
        [1, 10, 1],
        [2, 3, -1],
        [2, 4, 1],
        [2, 5, -1],
        [2, 6, -1],
        [2, 7, -1],
        [2, 8, 1],
        [2, 9, -1],
        [2, 10, -1],
        [3, 4, 1],
        [3, 5, -1],
        [3, 6, 1],
        [3, 7, 1],
        [3, 8, 1],
        [3, 9, -1],
        [3, 10, 1],
        [4, 5, -1],
        [4, 6, -1],
        [4, 7, -1],
        [4, 8, 1],
        [4, 9, -1],
        [4, 10, -1],
        [5, 6, -1],
        [5, 7, 1],
        [5, 8, 1],
        [5, 9, 1],
        [5, 10, 1],
        [6, 7, 1],
        [6, 8, 1],
        [6, 9, 1],
        [6, 10, 1],
        [7, 8, 1],
        [7, 9, -1],
        [7, 10, 1],
        [8, 9, -1],
        [8, 10, -1],
        [9, 10, 1],
    ]
)

# Teams are 1-indexed so use 0-indexed instead to play nice with python
train[:, 0] -= 1
train[:, 1] -= 1
test[:, 0] -= 1
test[:, 1] -= 1

### Run optimisation

First we form the _game incidence matrix_ A as suggested in the question, using a scipy spare
matrix in coordinate format.

In [3]:
team_1 = train[:, 0]
team_2 = train[:, 1]
outcome = train[:, 2]
i = np.arange(m)

A = (
    scipy.sparse.coo_matrix((outcome, (i, team_1)), shape=(m, n))
    + scipy.sparse.coo_matrix((-outcome, (i, team_2)), shape=(m, n))
)

Each time has an modelled ability $a_i \in [0, 1]$, such that when teams $i$ and $j$ play each
other, the probability that team $i$ wins is equal to $\mathbb{P}(a_i - a_j + v > 0)$ where
$v \sim \mathcal{N}(0, \sigma^2)$.

We want to find the maximum likelihood estimate of the abilities $a_i$.

What is the likelihood for match $i$ between teams $j$ and $k$ with outcome $v$?

$$
\mathcal{L}(a_j, a_k | v) = \left\{
\begin{array}{ll}
       \mathbb{P}(v > a_j - a_i) & v = 1 \\
       \mathbb{P}(v \leq a_j - a_i) & v = -1 \\
\end{array} 
\right.
= \left\{
\begin{array}{ll}
       \Phi(a_i - a_j) & v = 1 \\
       \Phi(a_j - a_i) & v = -1 \\
\end{array} 
\right.
$$

where $\Phi$ is the CDF of the normal distribution $\mathcal{N}(0, \sigma^2)$.

To get the total (log) likelihood, we sum the log of the the individual match likelihoods.

In [4]:
a = cp.Variable(n)

objective = cp.Maximize(cp.sum(cp.log_normcdf(A @ a / sigma)))
constraints = [a >= 0, a <= 1]

problem = cp.Problem(objective, constraints)

problem.solve()

-11.430294646964965

How does our solution compare to the test set? First of all, we can get a baseline by just guessing the same results
as in `train`.

In [5]:
# Assert the games are in the same order so we can directly compare the outcomes
assert (train[:, :-1] == test[:, :-1]).all()

percent_correct = (train[:, -1] == test[:, -1]).mean()
print(f"Correct prediction when using the same outcome as train: {percent_correct * 100:.1f}%")

Correct prediction when using the same outcome as train: 75.6%


In [6]:
team_1 = test[:, 0]
team_2 = test[:, 1]
i = np.arange(m)

A_test = (
    scipy.sparse.coo_matrix((np.ones(m), (i, team_1)), shape=(m, n))
    + scipy.sparse.coo_matrix((-np.ones(m), (i, team_2)), shape=(m, n))
)

predictions = np.where(A_test @ a.value >= 0, 1, -1)
percent_correct = (predictions == test[:, -1]).mean()
print(f"Correct prediction when using modelled abilities: {percent_correct * 100:.1f}%")

Correct prediction when using modelled abilities: 86.7%


# Question 3: Allocation of interdiction effort 

### Convert Matlab data generation to numpy

In [7]:
base_filepath = 'data/homework_9/interdiction'
n = 10
m = 20

edges_df = pd.read_csv(f'{base_filepath}/edges.csv', header=None, prefix='e_')
edges = np.array(edges_df)
# Convert to 0 index
edges -= 1

A_df = pd.read_csv(f'{base_filepath}/A.csv', header=None, prefix='A_')
A = np.array(A_df)

a_df = pd.read_csv(f'{base_filepath}/a_small.csv', header=None, prefix='A_')
a = np.array(a_df)

x_max_df = pd.read_csv(f'{base_filepath}/x_max.csv', header=None, prefix='xm_')
x_max = np.array(x_max_df)

B = m / 2

### Run optimisation

Consider the recursive problem in the question. We have that
$$
P_i = max_{j \in parents}(e^{- a_{ij}x_{ij}}P_j)
$$

or, writing $z_i = logP_i$, 
$$
z_i = max_{j \in parents}(z_j - a_{ij}x_{ij})
$$

Or
$$
z_i >= (z_j - a_{ij}x_{ij}) \forall j \in parents
\implies
z_i - z_j >= - a_{ij}x_{ij} \forall j \in parents
$$

This final condition is just $A^Tz \succcurlyeq a * x$ where $*$ denotes elementwise multiplication.

In [8]:
# Budget for each edge
x = cp.Variable((m,1))

# Log failure probabilities
z = cp.Variable((n,1))

objective = cp.Minimize(z[-1])
constraints = [
    0 <= x,
    x <= x_max,
    cp.sum(x) <= B,
    z[0] == 0,
    A.T @ z >= - cp.multiply(a, x)
]

z_star = cp.Problem(objective, constraints).solve()

print(f"Minimised P_max: {np.exp(z_star):.3f}")

Minimised P_max: 0.043


For uniform allocation of resources, we just set x to be uniform (and drop its constraints)

In [9]:
# Budget for each edge
x = np.ones(m) * B / m

# Log failure probabilities
z = cp.Variable(n)

objective = cp.Minimize(z[-1])
constraints = [
    z[0] == 0,
    A.T @ z >= - cp.diag(a) @ x
]

z_star = cp.Problem(objective, constraints).solve()

print(f"Minimised P_max: {np.exp(z_star):.3f}")

Minimised P_max: 0.247
